In [34]:
try:
    import Image
except ImportError:
    from PIL import Image
import pytesseract
import webbrowser
import pyscreenshot as ImageGrab
from googleapiclient.discovery import build
import pprint
import cPickle as pickle
from pynput.keyboard import Key, Controller
import time
import re
import tweepy
import csv
import pandas as pd

In [23]:
access_token, access_token_secret = pickle.load(open('token_secret.p', 'rb'))

In [24]:
consumer_key, consumer_secret = pickle.load(open('key_secret.p', 'rb'))

In [26]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

# public_tweets = api.home_timeline()
# for tweet in public_tweets:
#     print tweet.text

In [32]:
def get_all_tweets(screen_name):
	#Twitter only allows access to a users most recent 3240 tweets with this method
	
	#authorize twitter, initialize tweepy
# 	auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# 	auth.set_access_token(access_key, access_secret)
# 	api = tweepy.API(auth)
	
	#initialize a list to hold all the tweepy Tweets
	alltweets = []	
	
	#make initial request for most recent tweets (200 is the maximum allowed count)
	new_tweets = api.user_timeline(screen_name = screen_name,count=200)
	
	#save most recent tweets
	alltweets.extend(new_tweets)
	
	#save the id of the oldest tweet less one
	oldest = alltweets[-1].id - 1
	
	#keep grabbing tweets until there are no tweets left to grab
	while len(new_tweets) > 0:
		print "getting tweets before %s" % (oldest)
		
		#all subsiquent requests use the max_id param to prevent duplicates
		new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
		
		#save most recent tweets
		alltweets.extend(new_tweets)
		
		#update the id of the oldest tweet less one
		oldest = alltweets[-1].id - 1
		
		print "...%s tweets downloaded so far" % (len(alltweets))
	
	#transform the tweepy tweets into a 2D array that will populate the csv	
	outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]
	
	#write the csv	
	with open('%s_tweets.csv' % screen_name, 'wb') as f:
		writer = csv.writer(f)
		writer.writerow(["id","created_at","text"])
		writer.writerows(outtweets)
	
	pass

In [33]:
get_all_tweets('HQTriviaScribe')

getting tweets before 1016895949413773311
...400 tweets downloaded so far
getting tweets before 1013997546346303487
...600 tweets downloaded so far
getting tweets before 1009876751927955455
...800 tweets downloaded so far
getting tweets before 1004076533500252160
...1000 tweets downloaded so far
getting tweets before 1000090782920007679
...1200 tweets downloaded so far
getting tweets before 996234865996681215
...1400 tweets downloaded so far
getting tweets before 992934624765595647
...1600 tweets downloaded so far
getting tweets before 989581569156374530
...1800 tweets downloaded so far
getting tweets before 984598835145080831
...2000 tweets downloaded so far
getting tweets before 980251608100626431
...2200 tweets downloaded so far
getting tweets before 977259836139294721
...2400 tweets downloaded so far
getting tweets before 973999068765675525
...2600 tweets downloaded so far
getting tweets before 966782772676382720
...2800 tweets downloaded so far
getting tweets before 96287198588060

In [90]:
df = pd.read_csv('HQTriviaScribe_tweets.csv')

In [91]:
df['tweet_list'] = [re.sub(r'.\) ', '', tweet).split('\n') for tweet in df['text']]
print df.info()
#\xe2|\x9c|\x93

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2949 entries, 0 to 2948
Data columns (total 4 columns):
id            2949 non-null int64
created_at    2949 non-null object
text          2949 non-null object
tweet_list    2949 non-null object
dtypes: int64(1), object(3)
memory usage: 92.2+ KB
None


In [92]:
def label_answer(row):
    if len(row['tweet_list']) != 4:
        return -1
    if '\xe2\x9c\x93' in row['tweet_list'][1]:
        return 0
    elif '\xe2\x9c\x93' in row['tweet_list'][2]:
        return 1
    else:
        return 2
    
df['correct_answer'] = df.apply(lambda row: label_answer(row),axis=1)

df['question'] = [tweet[0] for tweet in tweet_list]
print df['question'][0]


print df.head()

#Problem: want to have correct answer in database. don't want to do for loops.

An 1800s scam involving watches led directly to the founding of which of these companies?
                    id           created_at  \
0  1019753041464119296  2018-07-19 01:17:25   
1  1019752695077404673  2018-07-19 01:16:03   
2  1019752397919449089  2018-07-19 01:14:52   
3  1019752231330041861  2018-07-19 01:14:12   
4  1019752059174780929  2018-07-19 01:13:31   

                                                text  \
0  An 1800s scam involving watches led directly t...   
1  A character named Orc appears as a protagonist...   
2  An original co-host of “20/20” was also the ed...   
3  Which car company founder once held the world ...   
4  Which of these polygons has the most sides?\n1...   

                                          tweet_list  correct_answer  \
0  [An 1800s scam involving watches led directly ...               0   
1  [A character named Orc appears as a protagonis...               1   
2  [An original co-host of “20/20” was also the e...               0   
3 

In [111]:
print df['text'][3]
df2 = df[df['correct_answer'] != -1]
print df2['correct_answer'][10]
df2['answer0'] = [re.sub(' \xe2|\x9c|\x93', '', tl[1]) for tl in df2['tweet_list']]
df2['answer1'] = [re.sub(' \xe2|\x9c|\x93', '', tl[2]) for tl in df2['tweet_list']]
df2['answer2'] = [re.sub(' \xe2|\x9c|\x93', '', tl[3]) for tl in df2['tweet_list']]

print df2.loc[0]

Which car company founder once held the world land speed record for wheel-driven vehicles?
1) Henry Ford ✓
2) Ferru… https://t.co/m5Gb77cODO
0


/Users/ketanagrawal/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/ketanagrawal/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/ketanagrawal/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [120]:
#df2.loc[0]
#df2.drop(['text', 'tweet_list', 'created_at'], axis=1, inplace=True)
print df2.head()
print df2['question'][5]
df2.to_csv('HQquestions_clean.csv')

                    id  correct_answer  \
0  1019753041464119296               0   
1  1019752695077404673               1   
2  1019752397919449089               0   
4  1019752059174780929               1   
5  1019751848176218113               0   

                                            question      answer0  \
0  An 1800s scam involving watches led directly t...        Sears   
1  A character named Orc appears as a protagonist...  JRR Tolkien   
2  An original co-host of “20/20” was also the ed...      Esquire   
4        Which of these polygons has the most sides?  Trisdecagon   
5  Sailors once gave avocados the nickname of “mi...       Butter   

         answer1       answer2  
0     Victorinox  Kay Jewelers  
1  William Blake  Ray Bradbury  
2    Vanity Fair          Time  
4   Enneadecagon   Octadecagon  
5          Jewel           Nut  
Sailors once gave avocados the nickname of “midshipman’s” what?


In [2]:
my_api_key, my_cse_id = pickle.load(open('api_key_cse_id.p', 'rb'))
#to use Google Search custom API, you'll need your own API key and CSE ID.

def process(res_items):
    text_list = [re.sub(r'\n|,|:|\||\xa0|\s*\.+', '', item['title'] + ' ' + item['snippet']).lower() for item in res_items]
    return ' '.join(text_list)

def google_search(search_term, **kwargs):
    service = build("customsearch", "v1", developerKey=my_api_key)
    res = service.cse().list(q=search_term, cx=my_cse_id, **kwargs).execute()
    return process(res['items']), int(res['searchInformation']['formattedTotalResults'].replace(',', ''))

def print_results(results):
    for result in results:
        for key in result:
            print '%s: %s' % (key, result[key])
        print '\n\n'

In [3]:
print google_search('dogs', num=10)

(u"dog - wikipedia the domestic dog is a member of the genus canis (canines) which forms part of the wolf-like canids and is the most widely abundant terrestrial carnivore complete guide to caring for dogs  dog breed information dog your dog's online owners manual featuring articles about breed information dog selection training grooming and care for dogs and puppies funny dogs who don't want to take a bath - try not to laugh dec 10 2016 funny dog videos of funny dogs who don't want to take a bath or dogs and puppies who hate baths try not to laugh challenge - try not to laugh dog dog breeds adoption bringing a dog home and care everything you need to know about dogs including dog breeds how to adopt a dog bringing a dog home dog health and care and more! dogs dogs the adorable domestic pet beloved by millions the dog is one of the most popular pets in the world -- it is of course commonly referred to as \u201cman's best your d&d game doesn't have enough dogs in it - polygon 11 hours a

In [84]:
#Save for later: switch control

keyboard = Controller()
keyboard.press(' ')
keyboard.release(' ')

In [ ]:
''' coordinates for OCR text extraction from Reflector 3 window mirroring iPhone at "Fill Screen" scale,
positioned at (0, 0) on 13 inch MBA '''

question_coords = (55, 253, 376, 357)

answerX1, answerX2 = 75, 355
answerY1, answerY2 = 390, 415
offset = 63
answers_coords = [(answerX1, answerY1, answerX2, answerY2), 
                  (answerX1, answerY1 + offset, answerX2, answerY2 + offset),
                  (answerX1, answerY1 + 2*offset, answerX2, answerY2 + 2*offset)]

def parseQuestionAnswer():
    question_img=ImageGrab.grab(bbox=question_coords) # X1,Y1,X2,Y2
    answer_imgs = [ImageGrab.grab(bbox=coords) for coords in answers_coords]
    question = pytesseract.image_to_string(question_img).replace('\n', ' ').lower()
    answers = [pytesseract.image_to_string(img).lower() for img in answer_imgs]
    return question, answers

In [13]:
#every time a new question appears: 

'''
1. raw question
2. - 4. question + answers (no quotes)
'''
#For testing purposes
# question = 'Which of these is a species of snake?'
# answers = ['Ball python', 'Marbled salamander', 'Axolotl']
question = 'Rapa Nui is the indigenous name of what location?'.lower()
answers = [answer.lower() for answer in ['Greenland', 'Mount Everest', 'Easter Island']]


def getBestAnswer():
    #question, answers = parseQuestionAnswer()
    q_search = google_search(question, num=10)
    combine_queries = ['%s \"%s\"' % (question, answer) for answer in answers]
    combine_searches = [google_search(query, num=10) for query in combine_queries]
    answerCounts = [q_search[0].count(answer) for answer in answers]
    print answerCounts
    numSearchResults = [search[1] for search in combine_searches]
    print numSearchResults

In [14]:
start = time.time()
getBestAnswer()
end = time.time()

print end - start

[0, 0, 14]
[24600, 4500, 23900]
3.50920295715


In [6]:
while True:
    raw_input('press any key to grab answer:')
    getBestAnswer()

KeyboardInterrupt: 